# Binning SNR and Chastity data to generate Qtable 

**Concept**
<br>
The  main concept of this study is to utilize the approach of binning the SNR and chastity data to generate Qscore (or Qtable) of each channel (A, T, G, C) at each cycle. Qscore is the quality score of basecall. In this study, three variables were used and they are SNR, chastity and cycle. Qscore is generated from error rate and error rate is determined by bad reads/(good reads + bad reads). The definition of good and bad reads in this study is defined as aligned and not aligned reads, respectively. The files used for determining if the reads are aligned are the perfect match files. 

There are two studies in this file. The first study (section 2) is binning the data of selected cycles and this section of code was developed to do the binning of multiple experiments so that a lot of data were used to generate the Qtable for cycle 21 and 50. The second study (section 3) is binning the data from cycle 21 to 150. The first 20 cycles were excluded due to many junk clusters being presented. Only one experiment was conducted in the second approach due to the lengthy computational time. 

There is only one method to bin the chastity data but there are two binning methods for SNR. The reason why there are two binning methods for SNR is because SNR data has not been processed in a way that the data is relative to other channels. On the other hand, chastity data has been normalized and compared to the other channels; therefore, binning chastity uses the raw data generated from the bioinformatics pipeline. For SNR, the two binning methods are based on (1) normalized values and (2) basecall normalized values minus the next maximum value (max_minus). The normalized values are equal to raw values divided by the median of the raw values. The following example illustrates how the max_minus approach is done: let assume a cluster at a particular cycle has the normalized SNR values as A = 2.5, T= 3.5, G = 3.4, C = 2.8. The basecall for that cycle is G. In this example, the max_minus will be 3.4 (basecall normalized value) - 3.5 (next maximum value) = -0.1.

The binning methodology used in this study is different from the most common binning method, which is based on dividing the range of values in the data set to the equal section. For example, if a data set ranges from 0 to 100 and the bin edges will be 0, 20, 40, 60, 80 and 100 to generate five bins in this data set. However, the binning method of this study is based on equal population. For example, a data set has 100 data points and to generate 5 bins, the data set will first need to be sorted in the ascending order and the values corresponding to the positions (or indexes) of 1, 20, 40, 60, 80 and 100 in this data set are the bin edges. Here are the outlines for this file:
1. Import the python packages
2. Binning SNR and chastity data of selected cycles<br>
    2a. Function scripts<br>
    2b. Get bin edges<br>
    2c. Generate Qtable<br>
3. Binning all CRT87x SNR and CRFL data<br>
    3a. Function scripts<br>
    3b. Get bin edges<br>
    3c. Generate Qtable<br>

## (1) Import the python packages
First of all, import all athe requried python packages

In [69]:
import random, os, re, sys, operator, glob,math
import statistics as stat
import numpy as np
import seaborn as sns
import pandas as pd
from plotnine import *
from pathlib import Path
import natsort as ns
import itertools as it

## (2) Binning SNR and chastity data of selected cycles

### (2a) Function scripts

Function scripts for further analysis. Just run the cell and don't need to change anything 

In [2]:
def list_files(path, pattern):
    files_in_dir = []
    # r=>root, d=>directories, f=>files
    for r, d, f in os.walk(path):
       for item in f:
          if pattern in item:
             files_in_dir.append(os.path.join(r, item))
    return files_in_dir

def Bin_Equal_Count(dblist, Base, cycle, num_bin = 50):
    histdb = pd.concat(dblist).sort_values()
    rowvalue = len(histdb)/num_bin
    
    row = 0
    bin_edge = []
    while row < num_bin:
        rows = round(rowvalue*row)
        bin_edge.append(histdb.iloc[rows, ])
        row = row + 1
        
    bin_edge.append(histdb.iloc[(len(histdb))-1,])
    binname = Base + str(cycle)
    binedge_dict =  {binname : bin_edge}
    return binedge_dict

def dictionOne(L):
    result = {}
    for d in L:
        result.update(d)
    return result

def histdict(db, cycles = [21, 50]):
    hist_ID_list = []
    for B in ['A', 'T', 'G', 'C']:
        for i in cycles:
            hist_ID_list.append(B + str(i))
            
    hist_bin = { i : [] for i in hist_ID_list}
    
    for ID in hist_ID_list:
        x = db[ID].tolist()
        for i in range(0, len(x)-1):
            hist_bin[ID].append((x[i], x[i + 1]))
    return hist_bin

def make_folder(save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    

### (2b) Get bin edges

The following cell is for files and parameter input. Here are the description of each input:
1. experiment_folder_path = the folder path where all the raw data is stored.
2. save_edges_folder = the bin edges files will be saved
2. file_type_list = there should be six file types. When this study was conducted, these files had the following file extensions:<br>
    A. id.Base = it contains cluster ID, cycle number, basecall and the column headings are [ID, Cycle, Base]<br>
    B. id.miss = it contains cluster ID and the cycle numbers where the clusters become no longer perfect aligned to the reference genome. The column headings are [ID, Miss]<br>
    C. id.intensity", "id.score", "id.Signal", "id.SNR = background fluorescence intensity (intensity), chasity (score), relative fluorescence intensity (Signal), signal to noise ratio (SNR). The column headings are [ID, Cycle, A,	T, G, C]<br>
3. selected_cycle = enter the cycle numbers that will be analyzed as a list (e.g. [21, 50])
4. num_bin = numbers of bin

In [65]:
#input the file paths
experiment_folder_path = "D:/Binning_study_KW/Example/"
save_edges_folder = experiment_folder_path  + "Bin_edges/"
filetypelist = ["id.Base", "id.intensity", "id.score", "id.Signal", "id.SNR", "miss"]
selected_cycle = [21, 50]
num_bin = 40

The following two cells are the scripts to generate chastity and SNR bins. Three files will be generated after running these two scripts and they are the bin edges files for chastity, normalized SNR, and max_minus of normalized SNR. These files will be in a Bin_edges folder inside the folder defined in the experiment_folder_path.

In [66]:
#Get bin edges for chastity 

#input the data path
experiment_folder_path = experiment_folder_path

#generate the file list 
filetypelist = filetypelist
goodfiletype= ["good." + i for i in filetypelist[0:5]]

# Bin for chastity
goodfile = list_files(path = experiment_folder_path , pattern = goodfiletype[2])
goodID = list_files(path = experiment_folder_path , pattern = goodfiletype[0])

#get bin edge values
chas_value_dict_list = []
for B in ['A', 'T', 'G', 'C']:  
    for cyc in selected_cycle:
        value_list = []
        for file in range(0, len(goodfile)):
        
            IDdb = pd.read_csv(goodID[file], sep = '\t')
            db = pd.read_csv(goodfile[file], sep = '\t')
            
            db1 = pd.merge(IDdb, db, on = ["ID", "Cycle"], how = "inner")\
                            .rename(columns = {'Base' : 'call_base'})
                            
            db1 = db1[(db1['Cycle'] == cyc) & (db1['call_base'] == B)] 
            value_list.append(db1[B])              
        #create dictionary and append to value_dict_list            
        value_dict = Bin_Equal_Count(dblist = value_list, 
                                 cycle= cyc, 
                                 Base = B, num_bin = num_bin)
        
        chas_value_dict_list.append(value_dict)

#convert bin edges list to bin edges dataframe and save the folder        
chasdb = pd.DataFrame(dictionOne(chas_value_dict_list))
save_folder = save_edges_folder
make_folder(save_folder)
chasdb.to_csv(save_folder + 'Chastity_bin_edges_by_cycles_equal_count_' + str(num_bin) + '.tsv',
       sep='\t', index=False)        

In [67]:
#Get bin edges for SNR

#input the data path
experiment_folder_path = experiment_folder_path

filetypelist = filetypelist
goodfiletype= ["good." + i for i in filetypelist[0:5]]

goodfile = list_files(path = experiment_folder_path, pattern = goodfiletype[4])
goodID = list_files(path = experiment_folder_path, pattern = goodfiletype[0])

value_dict_list = []
max_dict_list = []

#get bin edge values
for B in ['A', 'T', 'G', 'C']:  
    for cyc in [21, 50]:
        value_list = []
        max_list = []
        for file in range(0, len(goodfile)):
        
            IDdb = pd.read_csv(goodID[file], sep = '\t')
            db = pd.read_csv(goodfile[file], sep = '\t')
            
            db1 = pd.merge(IDdb, db, on = ["ID", "Cycle"], how = "inner")\
                            .rename(columns = {'Base' : 'call_base'})
                            
            db1['A'] =  db1['A']/stat.median( db1['A'])
            db1['T'] =  db1['T']/stat.median( db1['T'])
            db1['G'] =  db1['G']/stat.median( db1['G'])
            db1['C'] =  db1['C']/stat.median( db1['C'])
            #To calculate the normalized value and append the values to value_list                
            db1 = db1[(db1['Cycle'] == cyc) & (db1['call_base'] == B)]                
            value_list.append(db1[B])
            
            #To calculate the maxminus value and append the values to max_list
            secondmax = db1[B] - db1[['A', 'T', 'G', 'C']]\
                                            .drop([B], axis=1)\
                                            .max(axis=1) 
            max_list.append(secondmax)
        
        #create dictionary and append to value_dict_list
        value_dict = Bin_Equal_Count(dblist = value_list, 
                                     cycle= cyc, 
                                     Base = B, num_bin = num_bin)
        #create maxminus dictionary and append to value_dict_list
        max_dict = Bin_Equal_Count(dblist = max_list, 
                                     cycle= cyc, 
                                     Base = B, num_bin = num_bin)
        
        value_dict_list.append(value_dict)
        max_dict_list.append(max_dict)
#concatenate dictionary to dataframe and save the bin edges
SNRvaluedb = pd.DataFrame(dictionOne(value_dict_list))
SNRmaxdb = pd.DataFrame(dictionOne(max_dict_list))
make_folder(save_folder)
save_folder = save_edges_folder

make_folder(save_folder)
SNRvaluedb.to_csv(save_folder + 'SNR_value_bin_edges_by_cycles_equal_count_' + str(num_bin) + '.tsv',
       sep='\t', index=False)  

SNRmaxdb.to_csv(save_folder + 'SNR_max_diff_bin_edges_by_cycles_equal_count_' + str(num_bin) + '.tsv',
       sep='\t', index=False)       

### (2c) Generate Qtable

After getting the bin edges, run the following two cells to generate the Qtable. Here are the path information:
1. experiment_folder_path = it should be the same as the experiment_folder_path in the section (2a-2), where all the data is stored
2. result_folder = it is the folder you want to store your results
3. SNR_bin_edge_path = it is the SNR bin edges files generated using normalized value
4. SNR_bin_edge_max_path = it is the SNR bin edges files generated using maxminus method
5. SNR_bin_edge_max_path = it is the chastity bin edges files
6. selected cycles = the cycles will be analyzed and in this particular study, the cycles are [21, 50]
7. chastity_reduce_bin = the numbers of chastity bin will be compressed into one bin 
8. SNR_reduce_bin = the numbers of SNR bin will be compressed into one bin  

In [72]:
#input path, selected_cycle, number of chastity and SNR bin
experiment_folder_path = "D:/Binning_study_KW/Example/"
result_folder = 'D:/Binning_study_KW/Example/Result/'
SNR_bin_edge_path = 'D:/Binning_study_KW/Example/Bin_edges/SNR_value_bin_edges_by_cycles_equal_count_30.tsv'
SNR_bin_edge_max_path = 'D:/Binning_study_KW/Example/Bin_edges/SNR_max_diff_bin_edges_by_cycles_equal_count_30.tsv'
chasity_bin_edge_path = 'D:/Binning_study_KW/Example/Bin_edges/Chastity_bin_edges_by_cycles_equal_count_30.tsv'
selected_cycle = [21, 50]
chastity_reduce_bin = 10
SNR_reduce_bin = 4

After entering the path and other input, run the following cell to generate the results

In [71]:
# this is the scripts to generate the numbers of good and bad reads, error rate and Qscores

#load the raw data files
filetypelist = ["id.Base", "id.intensity", "id.score", "id.Signal", "id.SNR", "miss"]

goodfiletype= ["good." + i for i in filetypelist[0:5]]
badfiletype= ["bad." + i for i in filetypelist]

goodchasfile = list_files(path = experiment_folder_path, pattern = goodfiletype[2])
badchasfile = list_files(path = experiment_folder_path, pattern = badfiletype[2])

goodSNRfile = list_files(path = experiment_folder_path, pattern = goodfiletype[4])
badSNRfile = list_files(path = experiment_folder_path, pattern = badfiletype[4])

goodID = list_files(path = experiment_folder_path, pattern = goodfiletype[0])
badID = list_files(path = experiment_folder_path, pattern = badfiletype[0])

badmissfile = list_files(path = experiment_folder_path, pattern = badfiletype[5])

filename = list(map(lambda x: re.findall(r"CRT\d+x_Inc_\w\d\d\.\d\d", x), goodID))
filename = [i[0] for i in filename]


#load bin edges files
SNR_db = pd.read_csv(SNR_bin_edge_path, sep = "\t")
SNR_max_db = pd.read_csv(SNR_bin_edge_max_path, sep = "\t")
chas_db = pd.read_csv(chasity_bin_edge_path, sep = "\t")

# this dictionary is for looping later
SNR_hist_bin = {'SNR_value' : SNR_db, 'maxminus' : SNR_max_db}

db_dict = {}
for file in range(0, len(goodID)):
    Good_chas_list = []
    bad_chas_list_good = []
    bad_chas_list_bad = []
    Good_SNR_list = []
    bad_SNR_list_good = []
    bad_SNR_list_bad = []
    
    #Read the good and bad chastity files and rearange the data 
    for cyc in [21, 50]:
    
        #read the good_chastity, good ID files   
        IDdb = pd.read_csv(goodID[file], sep = '\t')
        db = pd.read_csv(goodchasfile[file], sep = '\t')
        
        db1 = pd.merge(IDdb, db, on = ["ID", "Cycle"], how = "inner")\
                        .rename(columns = {'Base' : 'call_base'})
                        
        db1 =  db1[db1.Cycle == cyc]     
                        
        db1['chas_value'] = db1.lookup(db1.index, db1['call_base'].astype(str))
        db1 = db1[['ID', 'Cycle', 'call_base', 'chas_value']]
        db1['type'] = 0 #all data is aligned (0)
        
        Good_chas_list.append(db1)
        
        #read the bad chastity and ID files and miss files    
        IDdb = pd.read_csv(badID[file], sep = '\t')
        db = pd.read_csv(badchasfile[file], sep = '\t')
        missdb = pd.read_csv(badmissfile[file], sep = '\t')
        
        db1 = pd.merge(IDdb, db, on = ["ID", "Cycle"], how = "inner")\
                        .rename(columns = {'Base' : 'call_base'})
        # remove all the reads contains N                
        db1 =  db1[(db1.call_base != "N") & (db1.Cycle == cyc)]                
                        
        db1['chas_value'] = db1.lookup(db1.index, db1['call_base'].astype(str))
        
        #this script is to label the ID as aligned (0) and not aligned (1) for cycle 21 and 50
        if cyc == 21:
            missdb.loc[missdb['Miss'] == 21, 'type'] = 1
            missdb.loc[missdb['Miss'] > 21, 'type'] = 0
            
            db2 = pd.merge(db1, missdb[['ID', 'type']], on = ["ID"], 
                           how = "inner")\
                            [['ID', 'Cycle', 'call_base', 'chas_value', 'type']] 
                            
            bad_chas_list_good.append(db2[db2['type'] == 0])
            bad_chas_list_bad.append(db2[db2['type'] == 1])
            
        elif cyc == 50:
            missdb = missdb[missdb.Miss == 50].assign(type  = 1)
            
            db2 = pd.merge(db1, missdb[['ID', 'type']], on = ["ID"], 
                           how = "inner")\
                            [['ID', 'Cycle', 'call_base', 'chas_value', 'type']]
            bad_chas_list_bad.append(db2[db2['type'] == 1])
            
        #Read the good SNR adn ID files, normalized the values and calculate maxminus 
        IDdb = pd.read_csv(goodID[file], sep = '\t')
        db = pd.read_csv(goodSNRfile[file], sep = '\t')
        
        db1 = pd.merge(IDdb, db, on = ["ID", "Cycle"], how = "inner")\
                        .rename(columns = {'Base' : 'call_base'})
                        
        db1 =  db1[db1.Cycle == cyc]
        
        db1['A'] =  db1['A']/stat.median( db1['A'])
        db1['T'] =  db1['T']/stat.median( db1['T'])
        db1['G'] =  db1['G']/stat.median( db1['G'])
        db1['C'] =  db1['C']/stat.median( db1['C'])
        
        db1['SNR_value'] = db1.lookup(db1.index, db1['call_base'].astype(str))
        #calculate the maxminus values
        m = db1.columns[3:7].to_numpy() == db1['call_base'].to_numpy()[:, None]
        db1['maxminus'] = db1['SNR_value'] - db1.drop(columns= ['ID', 'Cycle',
                                                                'call_base', 
                                                                'SNR_value'])\
                                                                  .mask(m).max(1)
        
        
        db1 = db1[['ID', 'Cycle', 'call_base', 'SNR_value', 'maxminus']]
        db1['type'] = 0
        
        Good_SNR_list.append(db1)
        
        #Read the good SNR adn ID files, normalized the values and calculate maxminus   
        IDdb = pd.read_csv(badID[file], sep = '\t')
        db = pd.read_csv(badSNRfile[file], sep = '\t')
        missdb = pd.read_csv(badmissfile[file], sep = '\t')
        
        db1 = pd.merge(IDdb, db, on = ["ID", "Cycle"], how = "inner")\
                        .rename(columns = {'Base' : 'call_base'})
                        
        db1 =  db1[(db1.call_base != "N") & (db1.Cycle == cyc)]  

        db1['A'] =  db1['A']/stat.median( db1['A'])
        db1['T'] =  db1['T']/stat.median( db1['T'])
        db1['G'] =  db1['G']/stat.median( db1['G'])
        db1['C'] =  db1['C']/stat.median( db1['C'])
                      
        db1['SNR_value'] = db1.lookup(db1.index, db1['call_base'].astype(str))
        m = db1.columns[3:7].to_numpy() == db1['call_base'].to_numpy()[:, None]
        db1['maxminus'] = db1['SNR_value'] - db1.drop(columns= ['ID', 'Cycle',
                                                                'call_base', 
                                                                'SNR_value'])\
                                                                  .mask(m).max(1)
        #this script is to label the ID as aligned (0) and not aligned (1) for cycle 21 and 50
        if cyc == 21:
            missdb.loc[missdb['Miss'] == 21, 'type'] = 1
            missdb.loc[missdb['Miss'] > 21, 'type'] = 0
            
            db2 = pd.merge(db1, missdb[['ID', 'type']], on = ["ID"], 
                           how = "inner")\
                            [['ID', 'Cycle', 'call_base', 'SNR_value', 
                              'type', 'maxminus']] 
                            
            bad_SNR_list_good.append(db2[db2['type'] == 0])
            bad_SNR_list_bad.append(db2[db2['type'] == 1])
            
        elif cyc == 50:
            missdb = missdb[missdb.Miss == 50].assign(type  = 1)
            
            db2 = pd.merge(db1, missdb[['ID', 'type']], on = ["ID"], 
                           how = "inner")\
                            [['ID', 'Cycle', 'call_base', 'SNR_value', 
                              'type', 'maxminus']]
            bad_SNR_list_bad.append(db2[db2['type'] == 1])
            
    #combine list of dataframe into one dataframe
    SNR_good_combine_list =  Good_SNR_list + bad_SNR_list_good
    chas_good_combine_list =  Good_chas_list + bad_chas_list_good
    
    SNR_good_combine_list = pd.concat(SNR_good_combine_list)
    chas_good_combine_list = pd.concat(chas_good_combine_list)
    
    SNR_bad_combine_list = pd.concat(bad_SNR_list_bad)
    chas_bad_combine_list = pd.concat(bad_chas_list_bad)
    
    #Adding bin ID to the combined dataframe, there are three loops: (1) it is to loop through two different SNR binning 
    #methods, (2) different bases, and (3) different cycles.
    for s in SNR_hist_bin.keys():
        save_list = []
        for B in ['A', 'T', 'G', 'C']:
            
            bin_id_list = []
            
            #Adding bin ID to the combined good reads dataframe
            for cyc in selected_cycle:
                dfm = pd.merge(SNR_good_combine_list\
                               [[s, 'ID', 'Cycle']], 
                               chas_good_combine_list, 
                               on = ["ID", "Cycle"], how = "inner")
                df = dfm[['ID',  'Cycle', 'call_base',  'chas_value', 
                          s ,'type']][(dfm['call_base'] == B)\
                                                  & (dfm['Cycle'] == cyc)]
                
                hist_ID = B + str(cyc)
                
                # create the bin ID column for chastity
                hist_bin = chas_db[hist_ID][0: len(chas_db[hist_ID])-chastity_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_C{i}' for i in range(1, len(bin_edges))]
                df['chas_bin'] =  pd.cut(x = df['chas_value'], bins = bin_edges, labels= values)

                # create the bin ID column for SNR
                hist_bin = SNR_hist_bin[s][hist_ID][0: len(SNR_hist_bin[s][hist_ID]) - SNR_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_S{i}' for i in range(1, len(bin_edges))]
                df['SNR_bin'] = pd.cut(x = df[s], bins = bin_edges, labels= values)

                df['bin_id'] = df.Cycle.astype(str) + df.chas_bin.astype(str) + df.SNR_bin.astype(str)
                bin_id_list.append(df['bin_id'])
                
                bin_db = pd.concat(bin_id_list)
                bin_dict = bin_db.value_counts().to_dict()
                
                ID_column = []
                for cyc in selected_cycle:
                    for c in range(1, len(bin_edges)):
                        for r in range(1, len(bin_edges)):
                            ID = str(cyc) + "_C" + str(c) + "_S" + str(r)
                            ID_column.append(ID)
                            
                count_column = [0] * len(ID_column)
                
            Good_bin_db = pd.DataFrame({'ID' : ID_column, 'good_counts' : count_column})
            Good_bin_db['good_counts'] = Good_bin_db['ID'].map(bin_dict).fillna(0)
            
            #Adding bin ID to the combined bad reads dataframe
            bin_id_list = []
            for cyc in [21, 50]:
            
                dfm = pd.merge(SNR_bad_combine_list\
                                    [[s, 'ID', 'Cycle']], 
                                    chas_bad_combine_list, 
                                    on = ["ID", "Cycle"], how = "inner")
                df = dfm[['ID',  'Cycle', 'call_base',  'chas_value', 
                       s ,'type']][dfm.call_base == B]
                
                hist_ID = B + str(cyc)
                
                # create the bin ID column for chastity
                hist_bin = chas_db[hist_ID][0: len(chas_db[hist_ID])- chastity_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_C{i}' for i in range(1, len(bin_edges))]
                df['chas_bin'] =  pd.cut(x = df['chas_value'], bins = bin_edges, labels= values)

                # create the bin ID column for SNR
                hist_bin = SNR_hist_bin[s][hist_ID][0: len(SNR_hist_bin[s][hist_ID])- SNR_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_S{i}' for i in range(1, len(bin_edges))]
                df['SNR_bin'] = pd.cut(x = df[s], bins = bin_edges, labels= values)

                df['bin_id'] = df.Cycle.astype(str) + df.chas_bin.astype(str) + df.SNR_bin.astype(str)
                bin_id_list.append(df['bin_id'])
                
                bin_db = pd.concat(bin_id_list)
                bin_dict = bin_db.value_counts().to_dict()
                
                ID_column = []
                for cyc in selected_cycle:
                    for c in range(1, len(bin_edges)):
                        for r in range(1, len(bin_edges)):
                            ID = str(cyc) + "_C" + str(c) + "_S" + str(r)
                            ID_column.append(ID)
                            
                count_column = [0] * len(ID_column)
            
            Bad_bin_db = pd.DataFrame({'ID' : ID_column, 'bad_counts' : count_column})
            Bad_bin_db['bad_counts'] = Bad_bin_db['ID'].map(bin_dict).fillna(0)
        
            #merging the good and bad reads together and append the dataframes to a dictionary 
            save_bin_db = pd.merge(Bad_bin_db, Good_bin_db, 
                           on = ["ID"], how = "inner")
            
            result_folder = result_folder
            make_folder(result_folder)
            
            list_name = filename[file] + '_' + s + '_base_' + B
            db_dict[list_name] = save_bin_db
            
# This section is to process db_list and save the files. Four types of files ["bad_counts", "good_counts", "error", "qscore"]
# at each cycle and bases will be saved. The file will be saved in the format of SNR bins as the row index and chastity bins
# as the column headings
chastity_name = ['chastity' + str(x) for x in range(1, len(bin_edges))]
SNR_name = ['SNR' + str(x) for x in range(1, len(bin_edges))]

for SNR_method in SNR_hist_bin.keys():
    for base in ["base_A", "base_T", "base_G", "base_C"]:
        L1 = [s for s in db_dict.keys() if SNR_method in s and base in s]
        L2 = []
        for file in L1:
            L2.append(db_dict[file])
        db = pd.concat(L2).groupby(by=["ID"]).sum().reset_index().sort_values(by=['ID'])
        db['ID'] = pd.Categorical(db['ID'], ordered=True, categories= ns.natsorted(db['ID'].unique()))
        db = db.sort_values('ID')
        db[['Cycle','chas_bin', 'SNR_bin']] = db.ID.str.split("_",expand=True,)
        db['error'] = db.bad_counts / (db.good_counts + db.bad_counts )
        db['qscore'] = round((10 * -np.log10(db.error)), 0)
        for cyc in selected_cycle:
            for name in ["bad_counts", "good_counts", "error", "qscore"]:
                db1 = db[db.Cycle == str(cyc)][name]
                n = len(bin_edges)-1
                outList = []
                for i in range(n, len(db1) + n, n):
                    outList.append(db1[i-n:i])

                db2 = pd.DataFrame(np.array(outList), columns=chastity_name,
                                  index = SNR_name)
                savefolder = result_folder + SNR_method + '/' + "cycle_" + str(cyc) + "_" +  base +  "/" 
                make_folder(savefolder)
                filename = savefolder + SNR_method + "_" + base + "_cycle_" + str(cyc) + "_" + name + '.csv'
                db2.to_csv(filename, sep=',', index=True)

## 3. Binning all CRT87x SNR and CRFL data 

### (3a) Function scripts

Function scripts for further analysis. Just run the cell and don't need to change anything. 

In [33]:
# make histogram ID list
def make_hist_ID(cycle):
    hist_ID_list = []
    for B in ['A', 'T', 'G', 'C']:
        for i in cycle:
            hist_ID_list.append(B + str(i))
    return hist_ID_list

# generate bin edges for SNR, CRFL, chastity
def get_bin_edges(raw_data_path, chunksize, selected_cycle, num_bin, 
                  Good_File_Extension, Good_ID_Extension, divided_median):
    filelist = []
    for file in os.listdir(raw_data_path):
        f = raw_data_path + file
        filelist.append(f)

    good_file_list = [s for s in filelist if Good_File_Extension in s]
    good_ID_list = [s for s in filelist if s.endswith(Good_ID_Extension)]

    hist_ID_list = make_hist_ID(selected_cycle)

    binedge_dict = { i : [] for i in hist_ID_list}
    count_dict = { i : [] for i in hist_ID_list}

    for B in ['A', 'T', 'G', 'C']:
        Good_chunk_list = []

        for file in range(0, len(good_file_list)):

            perfect = pd.read_csv(good_file_list[file], sep = "\t", usecols = ['ID', 'Cycle', B], 
                                      chunksize = chunksize)
            median = stat.median(pd.read_csv(good_file_list[file], 
                                      sep = "\t", usecols = [B])[B])
            dbperfect = pd.read_csv(good_ID_list[file],
                                        sep = "\t")

            for data_chunk in perfect:  

                data_chunk = data_chunk.sort_values(by=['ID', 'Cycle'])
                db1 = dbperfect[dbperfect['ID'].isin(data_chunk.ID.unique().tolist())]

                slist = ""
                for s in db1.Sequence[0:len(db1.Sequence)]:
                    slist = slist + str(s)

                baselist = list(slist)

                db2 = pd.DataFrame({'ID' : data_chunk.ID, 'Cycle' : data_chunk.Cycle, 'call_base' : baselist})  
                db3 = pd.merge(data_chunk, db2, on = ["ID", "Cycle"], how = "inner") 
                db3 = db3[db3.call_base == B]
                if divided_median == True:
                    db3[B] = db3[B]/median

                Good_chunk_list.append(db3[[B, "Cycle"]])

        histdb = pd.concat(Good_chunk_list)

        for cyc in selected_cycle:
            histdb2 = histdb[histdb.Cycle == cyc]
            histdb2 = histdb2[B].sort_values()
            rowvalue = len(histdb2)/num_bin

            row = 0
            bin_edge = []
            while row < num_bin:
                row_num = round(rowvalue*row)
                bin_edge.append(histdb2.iloc[row_num, ])
                row = row + 1

            bin_edge.append(histdb2.iloc[(len(histdb2))-1,])
            binname = B+str(cyc)
            binedge_dict[binname] =  bin_edge


    hist_binedge_db = pd.DataFrame(binedge_dict)
    return hist_binedge_db
#list all files in a folder
def list_files(path, pattern):
    files_in_dir = []
    # r=>root, d=>directories, f=>files
    for r, d, f in os.walk(path):
       for item in f:
          if pattern in item:
             files_in_dir.append(os.path.join(r, item))
    return files_in_dir
#get the bin edges using equal populations
def Bin_Equal_Count(dblist, Base, cycle, num_bin = 50):
    histdb = pd.concat(dblist).sort_values()
    rowvalue = len(histdb)/num_bin
    
    row = 0
    bin_edge = []
    while row < num_bin:
        rows = round(rowvalue*row)
        bin_edge.append(histdb.iloc[rows, ])
        row = row + 1
        
    bin_edge.append(histdb.iloc[(len(histdb))-1,])
    binname = Base + str(cycle)
    binedge_dict =  {binname : bin_edge}
    return binedge_dict

def dictionOne(L):
    result = {}
    for d in L:
        result.update(d)
    return result

def histdict(db, cycles = [21, 50]):
    hist_ID_list = []
    for B in ['A', 'T', 'G', 'C']:
        for i in cycles:
            hist_ID_list.append(B + str(i))
            
    hist_bin = { i : [] for i in hist_ID_list}
    
    for ID in hist_ID_list:
        x = db[ID].tolist()
        for i in range(0, len(x)-1):
            hist_bin[ID].append((x[i], x[i + 1]))
    return hist_bin

def make_folder(save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

### (3b) Get bin edges

The following cell is for files and parameter input. Here are the description of each input:
1. experiment_folder_path = the folder path where all the raw data is stored.
2. save_folder = the bin edges files will be saved
3. chunksize = due to large size data, the data will be processed in chunk.
4. num_bin = numbers of bin

In [3]:
experiment_folder_path = "D:/Binning_study_KW/Example_87x/Raw_data/"
save_folder = "D:/Binning_study_KW/Example_87x/Bin_edges/"
chunksize = 900000
selected_cycle = range(21, 151)
num_bin = 30

This cell is to generate bin edges for SNR normalized value and CRFL. "Good_ID.tsv.SNR" is for SNR and "Good_ID.tsv.intensity" is for CRFL. get_bin_edges function is to generate bin edges for these parameters. Due to SNR_maxminus is processed differently, get_bin_edges cannot generate bin edges for SNR_maxminus. The input of "divided_median" is the option for normalized the values by median. For SNR, the data is normalized but not for CRFL.

In [34]:
# Get SNR bin 
SNR_bin = get_bin_edges(raw_data_path = experiment_folder_path, 
              chunksize = chunksize, 
              selected_cycle = selected_cycle, 
              num_bin = num_bin,
              Good_File_Extension = "Good_ID.tsv.SNR", 
              Good_ID_Extension = 'Good_ID.tsv', 
              divided_median = True)
make_folder(save_folder)
save_name = 'CRT87x_SNR_bin_edges_by_cycles'
SNR_bin.to_csv(save_folder + save_name + '.tsv', sep='\t', index=False)

#Get CRFL bin
CRFL_bin = get_bin_edges(raw_data_path = experiment_folder_path, 
              chunksize = chunksize, 
              selected_cycle = selected_cycle, 
              num_bin = num_bin,
              Good_File_Extension = "Good_ID.tsv.intensity", 
              Good_ID_Extension = 'Good_ID.tsv', 
              divided_median = False)
make_folder(save_folder)
save_name = 'CRT87x_CRFL_bin_edges_by_cycles'
CRFL_bin.to_csv(save_folder + save_name + '.tsv', sep='\t', index=False)

Ths cell is to generate bin edges for SNR_maxminus 

In [42]:
#SNR_max_bin
filelist = []
for file in os.listdir(experiment_folder_path):
    f = experiment_folder_path + file
    filelist.append(f)

SNR_good_file_list = [s for s in filelist if "Good_ID.tsv.SNR" in s]
SNR_good_ID_list = [s for s in filelist if s.endswith('Good_ID.tsv')]

hist_ID_list = make_hist_ID(selected_cycle)
binedge_dict = { i : [] for i in hist_ID_list}

for B in ['A', 'T', 'G', 'C']:
    Good_SNR_chunk_list = []
    
    for file in range(0, len(SNR_good_file_list)):
        SNR_perfect = pd.read_csv(SNR_good_file_list[file], 
                          sep = "\t", usecols = ['ID', 'Cycle', 'A', 'T', 'G', 'C'], 
                          chunksize= chunksize)
        SNR_median_A = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['A'])['A'])
        SNR_median_T = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['T'])['T'])
        SNR_median_G = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['G'])['G'])
        SNR_median_C = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['C'])['C'])
        dbperfect = pd.read_csv(SNR_good_ID_list[file],
                                    sep = "\t")
        
        for data_chunk in SNR_perfect:  

            data_chunk = data_chunk.sort_values(by=['ID', 'Cycle'])
            
            db1 = dbperfect[dbperfect['ID'].isin(data_chunk.ID.unique().tolist())]
            
            slist = ""
            for s in db1.Sequence[0:len(db1.Sequence)]:
                slist = slist + str(s)
            
            baselist = list(slist)
            
            db2 = pd.DataFrame({'ID' : data_chunk.ID, 'Cycle' : data_chunk.Cycle, 
                                'call_base' : baselist})   
            db3 = pd.merge(data_chunk, db2, on = ["ID", "Cycle"], how = "inner") 
            db3['A'] = db3['A']/SNR_median_A
            db3['T'] = db3['T']/SNR_median_T
            db3['G'] = db3['G']/SNR_median_G
            db3['C'] = db3['C']/SNR_median_C
            db3 = db3[db3.call_base == B]
            db3['SNR_value'] = db3[B]
            m =db3.columns[2:6].to_numpy() == db3['call_base'].to_numpy()[:, None]
            db3['maxminus'] = db3['SNR_value'] - db3.drop(columns= ['ID', 'Cycle','call_base', 
                                                                    'SNR_value'])\
                                                                         .mask(m).max(1)
            
            Good_SNR_chunk_list.append(db3[['maxminus', 'Cycle']])

    histdb = pd.concat(Good_SNR_chunk_list).groupby(['Cycle'])

    
    for cyc in selected_cycle:
        histdb2 = histdb.get_group(cyc)
        histdb2 = histdb2['maxminus'].sort_values()
        rowvalue = len(histdb2)/num_bin
        
        row = 0
        bin_edge = []
        while row < num_bin:
            row_num = round(rowvalue*row)
            bin_edge.append(histdb2.iloc[row_num, ])
            row = row + 1

        bin_edge.append(histdb2.iloc[(len(histdb2))-1,])
        binname = B+str(cyc)
        binedge_dict[binname] =  bin_edge
    
    
hist_binedge_db = pd.DataFrame(binedge_dict)

save_folder = save_folder
make_folder(save_folder)

hist_binedge_db.to_csv(save_folder + 'CRT87x_SNR_maxminus_bin_edges.tsv',
       sep='\t', index=False)

### (3c) Generate Qtable

After getting the bin edges, run the following two cells to generate the Qtable. Here are the path information:
1. raw_data_path = it should be the same as the experiment_folder_path in the section (2a-2), where all the data is stored
2. result_folder = it is the folder you want to store your results
3. SNR_bin_edge_path = it is the SNR bin edges files generated using normalized value
4. SNR_bin_edge_max_path = it is the SNR bin edges files generated using maxminus method
5. SNR_bin_edge_max_path = it is the chastity bin edges files
6. selected cycles = the cycles will be analyzed and in this particular study
7. chunksize = number of row in a chunk; data is processed in chunk in order not to exceed the memory capacity.
7. chastity_reduce_bin = the numbers of chastity bin will be compressed into one bin 
8. SNR_reduce_bin = the numbers of SNR bin will be compressed into one bin  

In [125]:
raw_data_path = "D:/Binning_study_KW/Example_87x/Raw_data/"
result_folder = "D:/Binning_study_KW/Example_87x/Results/"
SNR_bin_edge_path = "D:/Binning_study_KW/Example_87x/Bin_edges/CRT87x_SNR_bin_edges_by_cycles.tsv.tsv"
SNR_bin_edge_max_path = "D:/Binning_study_KW/Example_87x/Bin_edges/CRT87x_SNR_maxminus_bin_edges.tsv"
chasity_bin_edge_path = "D:/Binning_study_KW/Example_87x/Bin_edges/CRT87x_CRFL_bin_edges_by_cycles.tsv.tsv"
selected_cycle = range(21, 151)
chunksize = 900000
chastity_reduce_bin = 0
SNR_reduce_bin = 0

Run the following cell to generate the results

In [ ]:
#load the raw data files
filelist = []
for fe in os.listdir(raw_data_path):
    f =raw_data_path + fe
    filelist.append(f)

SNR_good_file_list = [s for s in filelist if "Good_ID.tsv.SNRb" in s]
CRFL_good_file_list = [s for s in filelist if "Good_ID.tsv.intensity" in s]
good_ID_list = [s for s in filelist if s.endswith('Good_ID.tsv')]
SNR_bad_file_list = [s for s in filelist if "Bad_ID.tsv.SNRb" in s]
CRFL_bad_file_list = [s for s in filelist if "Bad_ID.tsv.intensity" in s]
bad_ID_list = [s for s in filelist if s.endswith('Bad_ID.tsv')]


filename = list(map(lambda x: re.findall(r"CRT87x_\w\d\d\.\d\d", x), bad_ID_list))
filename = [i[0] for i in filename]


#laod bin edges data
SNR_db = pd.read_csv(SNR_bin_edge_path, sep = "\t")
SNR_max_db = pd.read_csv(SNR_bin_edge_max_path, sep = "\t")
chas_db = pd.read_csv(chasity_bin_edge_path, sep = "\t")

SNR_hist_bin = {'SNR_value' : SNR_db, 'maxminus' : SNR_max_db}
db_dict = {}

#
for file in range(0, len(good_ID_list)):
    SNR_perfect = pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['ID', 'Cycle', 'A', 'T', 'G', 'C'], 
                              chunksize = chunksize)
    
    SNR_median_A = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['A'])['A'])
    SNR_median_T = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['T'])['T'])
    SNR_median_G = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['G'])['G'])
    SNR_median_C = stat.median(pd.read_csv(SNR_good_file_list[file], 
                              sep = "\t", usecols = ['C'])['C'])
    
    dbperfect = pd.read_csv(good_ID_list[file],
                              sep = "\t")
    #read the good_SNR, good ID files by chunk, normalized the values, calculate the maxminus and append the data
    # to Good_SNR_chunk_list
    Good_SNR_chunk_list = []
    for data_chunk in SNR_perfect:  
    
        data_chunk = data_chunk.sort_values(by=['ID', 'Cycle'])
        db1 = dbperfect[dbperfect['ID'].isin(data_chunk.ID.unique().tolist())]
        
        slist = ""
        for seq in db1.Sequence[0:len(db1.Sequence)]:
            slist = slist + str(seq)
        
        baselist = list(slist)
        
        db2 = pd.DataFrame({'ID' : data_chunk.ID, 'Cycle' : data_chunk.Cycle, 
                            'call_base' : baselist})   
        db3 = pd.merge(data_chunk, db2, on = ["ID", "Cycle"], how = "inner") 
        db3 = db3[['ID', 'Cycle', 'A', 'T', 'G', 'C', 'call_base']]
        db3['A'] = db3['A']/SNR_median_A
        db3['T'] = db3['T']/SNR_median_T
        db3['G'] = db3['G']/SNR_median_G
        db3['C'] = db3['C']/SNR_median_C
        db3['SNR_value'] = db3.lookup(db3.index, db3['call_base'].astype(str))
        m =db3.columns[2:6].to_numpy() == db3['call_base'].to_numpy()[:, None]
        db3['maxminus'] = db3['SNR_value'] - db3.drop(columns= ['ID', 'Cycle','call_base', 'SNR_value'])\
                                                                    .mask(m).max(1)
        db3 = db3[['ID', 'Cycle', 'call_base', 'SNR_value', 'maxminus']][db3.Cycle >20]
        db3['type'] = 0 #type indicates aligned (0) or not aligned (1) 
        
        Good_SNR_chunk_list.append(db3)
    
    
    #read the good_CRFL, good ID files by chunk and append the data to Good_CRFL_chunk_list
    CRFL_perfect = pd.read_csv(CRFL_good_file_list[file],       
                              sep = "\t", usecols = ['ID', 'Cycle', 'A', 'T', 'G', 'C'], 
                              chunksize=chunksize)
    dbperfect = pd.read_csv(good_ID_list[file],
                            sep = "\t")
    
    Good_CRFL_chunk_list = []
    
    for data_chunk in CRFL_perfect:  
    
        data_chunk = data_chunk.sort_values(by=['ID', 'Cycle'])
        db1 = dbperfect[dbperfect['ID'].isin(data_chunk.ID.unique().tolist())]
        
        slist = ""
        for seq in db1.Sequence[0:len(db1.Sequence)]:
            slist = slist + str(seq)
        
        baselist = list(slist)
        
        db2 = pd.DataFrame({'ID' : data_chunk.ID, 'Cycle' : data_chunk.Cycle, 
                            'call_base' : baselist})     
        db3 = pd.merge(data_chunk, db2, on = ["ID", "Cycle"], how = "inner") 
        db3['CRFL_value'] = db3.lookup(db3.index, db3['call_base'].astype(str))
        db3 = db3[['ID', 'Cycle', 'call_base', 'CRFL_value']][db3.Cycle >20]
        db3['type'] = 0 0 #type indicates aligned (0) or not aligned (1)
        
        
        Good_CRFL_chunk_list.append(db3)
        
    # read the bad_SNR, abd ID files by chunk, normalized the values, calculate the maxminus and append the aligned
    # data to Bad_SNR_chunk_list_good and the not aligned data to Bad_SNR_chunk_list_bad
    SNR_bad = pd.read_csv(SNR_bad_file_list[file], 
                              sep = "\t", usecols = ['ID', 'Cycle', 'A', 'T', 'G', 'C'], 
                              chunksize = chunksize)
    
    SNR_median_A = stat.median(pd.read_csv(SNR_bad_file_list[file], 
                              sep = "\t", usecols = ['A'])['A'])
    SNR_median_T = stat.median(pd.read_csv(SNR_bad_file_list[file], 
                              sep = "\t", usecols = ['T'])['T'])
    SNR_median_G = stat.median(pd.read_csv(SNR_bad_file_list[file], 
                              sep = "\t", usecols = ['G'])['G'])
    SNR_median_C = stat.median(pd.read_csv(SNR_bad_file_list[file], 
                              sep = "\t", usecols = ['C'])['C'])
    
    ID_db = pd.read_csv(bad_ID_list[file], sep = "\t")
    
    Bad_SNR_chunk_list_good = []
    Bad_SNR_chunk_list_bad =  []
    
    for data_chunk in SNR_bad:  
    
        data_chunk = data_chunk.sort_values(by=['ID', 'Cycle'])
        db1 = pd.merge(data_chunk, ID_db[['ID', 'Miss']], 
                       on = ["ID"], how = "inner").sort_values(by=['ID', 'Cycle'])
        
        db1['A'] = db1['A']/SNR_median_A
        db1['T'] = db1['T']/SNR_median_T
        db1['G'] = db1['G']/SNR_median_G
        db1['C'] = db1['C']/SNR_median_C
       
        ID_db2 = ID_db[ID_db['ID'].isin(db1.ID.unique().tolist())] 
        
        slist = ""
        for seq in ID_db2.Sequence[0:len(ID_db2.Sequence)]:
            slist = slist + str(seq)
        
        baselist = list(slist)
        
        db2 = pd.DataFrame({'ID' : data_chunk.ID, 'Cycle' : data_chunk.Cycle, 
                           'call_base' : baselist})    
        db3 = pd.merge(db1, db2, on = ["ID", "Cycle"], how = "inner") 
        db3 = db3[['ID', 'Cycle', 'A', 'T', 'G', 'C', 'Miss','call_base']]
        db3['SNR_value'] = db3.lookup(db3.index, db3['call_base'].astype(str))
        m =db3.columns[2:6].to_numpy() == db3['call_base'].to_numpy()[:, None]
        db3['maxminus'] = db3['SNR_value'] - db3.drop(columns= ['ID', 'Cycle','call_base', 'SNR_value', 'Miss'])\
                                                                  .mask(m).max(1)
        db4 = db3[db3.Cycle <= db3.Miss]
        db4['type'] = db4.Cycle == db4.Miss
        db4['type'] = db4['type'].astype(int)
        
        db5 = db4[['ID', 'Cycle', 'call_base', 'SNR_value', 'maxminus', 'type']][db4.Cycle >20]
        
        Bad_SNR_chunk_list_good.append(db5[db5.type == 0])
        Bad_SNR_chunk_list_bad.append(db5[db5.type == 1])
        
    # read the bad_CRFL, bad ID files by chunk and append the aligned
    # data to Bad_CRFL_chunk_list_good and the not aligned data to Bad_CRFL_chunk_list_bad
    CRFL_Bad = pd.read_csv(CRFL_bad_file_list[file], 
                              sep = "\t", usecols = ['ID', 'Cycle', 'A', 'T', 'G', 'C'], 
                              chunksize = chunksize)
    
    ID_db = pd.read_csv(bad_ID_list[file], sep = "\t")
        
    Bad_CRFL_chunk_list_good = []
    Bad_CRFL_chunk_list_bad =  []
    
    for data_chunk in CRFL_Bad:  
    
        data_chunk = data_chunk.sort_values(by=['ID', 'Cycle'])
        db1 = pd.merge(data_chunk, ID_db[['ID', 'Miss']], 
                       on = ["ID"], how = "inner").sort_values(by=['ID', 'Cycle'])
        
        ID_db2 = ID_db[ID_db['ID'].isin(db1.ID.unique().tolist())] 
        
        slist = ""
        for seq in ID_db2.Sequence[0:len(ID_db2.Sequence)]:
            slist = slist + str(seq)
        
        baselist = list(slist)
        
        db2 = pd.DataFrame({'ID' : data_chunk.ID, 'Cycle' : data_chunk.Cycle, 
                           'call_base' : baselist})   
        db3 = pd.merge(db1, db2, on = ["ID", "Cycle"], how = "inner") 
        db3 = db3[['ID', 'Cycle', 'A', 'T', 'G', 'C', 'Miss','call_base']]
        db3['CRFL_value'] = db3.lookup(db3.index, db3['call_base'].astype(str))
        db4 = db3[db3.Cycle <= db3.Miss]
        db4['type'] = db4.Cycle == db4.Miss
        db4['type'] = db4['type'].astype(int)
        
        db5 = db4[['ID', 'Cycle', 'call_base', 'CRFL_value', 'type']][db4.Cycle >20]
        
        Bad_CRFL_chunk_list_good.append(db5[db5.type == 0])
        Bad_CRFL_chunk_list_bad.append(db5[db5.type == 1])
        
    #append the aligned reads dataframe lists, concat to one data frame, group by cycle and call_base  
    SNR_good_combine_list =  Good_SNR_chunk_list + Bad_SNR_chunk_list_good
    CRFL_good_combine_list =  Good_CRFL_chunk_list + Bad_CRFL_chunk_list_good
    SNR_good_combine_list = pd.concat(SNR_good_combine_list).groupby(['Cycle', 'call_base'])
    CRFL_good_combine_list = pd.concat(CRFL_good_combine_list).groupby(['Cycle', 'call_base'])
    #concat the not aligned reads dataframe list and then group by cycle and call_base 
    Bad_CRFL_chunk_list_bad = pd.concat(Bad_CRFL_chunk_list_bad).groupby(['Cycle', 'call_base'])
    Bad_SNR_chunk_list_bad = pd.concat(Bad_SNR_chunk_list_bad).groupby(['Cycle', 'call_base'])
    
    goodkeylist = list(SNR_good_combine_list.groups.keys())
    badkeylist = list(Bad_CRFL_chunk_list_bad.groups.keys())
    # this loop is to bin the data and generate binning like Cycle#_C#_S#
    for s in SNR_hist_bin.keys():  
        for B in ['A', 'T', 'G', 'C']:
            filterkey = list(it.compress(goodkeylist, [ B in i for i in goodkeylist]))
            bin_id_list = []
            for key in filterkey: 

                dfm = pd.merge(SNR_good_combine_list.get_group(key)[[s, 'ID', 'Cycle']], 
                               CRFL_good_combine_list.get_group(key), 
                               on = ["ID", "Cycle"], how = "inner")
                df = dfm[['ID',  'Cycle', 'call_base',  'CRFL_value', s ,'type']]

                hist_ID = key[1] + str(key[0])

                 # create the bin ID column for chastity
                hist_bin = chas_db[hist_ID][0: len(chas_db[hist_ID])-chastity_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_C{i}' for i in range(1, len(bin_edges))]
                df['chas_bin'] =  pd.cut(x = df['CRFL_value'], bins = bin_edges, labels= values)

                # create the bin ID column for SNR
                hist_bin = SNR_hist_bin[s][hist_ID][0: len(SNR_hist_bin[s][hist_ID]) - SNR_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_S{i}' for i in range(1, len(bin_edges))]
                df['SNR_bin'] = pd.cut(x = df[s], bins = bin_edges, labels= values)

                df['bin_id'] = df.Cycle.astype(str) + df.chas_bin.astype(str) + df.SNR_bin.astype(str)
                bin_id_list.append(df['bin_id'])

            bin_db = pd.concat(bin_id_list)
            bin_dict = bin_db.value_counts().to_dict()

            ID_column = []
            for cyc in selected_cycle:
                for c in range(1, len(bin_edges)):
                    for r in range(1, len(bin_edges)):
                        ID = str(cyc) + "_C" + str(c) + "_S" + str(r)
                        ID_column.append(ID)

            count_column = [0] * len(ID_column)

            Good_bin_db = pd.DataFrame({'ID' : ID_column, 'good_counts' : count_column})
            Good_bin_db['good_counts'] = Good_bin_db['ID'].map(bin_dict).fillna(0)


            #Bad bin
            filterkey = list(it.compress(badkeylist, [ B in i for i in badkeylist]))
            bin_id_list = []
            for key in filterkey:     
                dfm = pd.merge(Bad_SNR_chunk_list_bad.get_group(key)[[s, 'ID', 'Cycle']], 
                               Bad_CRFL_chunk_list_bad.get_group(key), 
                               on = ["ID", "Cycle"], how = "inner")
                df = dfm[['ID',  'Cycle', 'call_base',  'CRFL_value', s ,'type']]

                hist_ID = key[1] + str(key[0])

                # create the bin ID column for chastity
                hist_bin = chas_db[hist_ID][0: len(chas_db[hist_ID])- chastity_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_C{i}' for i in range(1, len(bin_edges))]
                df['chas_bin'] =  pd.cut(x = df['CRFL_value'], bins = bin_edges, labels= values)

                # create the bin ID column for SNR
                hist_bin = SNR_hist_bin[s][hist_ID][0: len(SNR_hist_bin[s][hist_ID])- SNR_reduce_bin]
                bin_edges =  hist_bin[0: (len(hist_bin)-1)].tolist()
                bin_edges.append(float("inf")) #adding inf to last bin to capture all the upper values
                bin_edges[0] = -float("inf") #adding -inf to first bin to capture all the lower values
                values = [f'_S{i}' for i in range(1, len(bin_edges))]
                df['SNR_bin'] = pd.cut(x = df[s], bins = bin_edges, labels= values)

                df['bin_id'] = df.Cycle.astype(str) + df.chas_bin.astype(str) + df.SNR_bin.astype(str)
                bin_id_list.append(df['bin_id'])

            bin_db = pd.concat(bin_id_list)
            bin_dict = bin_db.value_counts().to_dict()

            ID_column = []
            for cyc in selected_cycle:
                for c in range(1, len(bin_edges)):
                    for r in range(1, len(bin_edges)):
                        ID = str(cyc) + "_C" + str(c) + "_S" + str(r)
                        ID_column.append(ID)

            count_column = [0] * len(ID_column)

            Bad_bin_db = pd.DataFrame({'ID' : ID_column, 'bad_counts' : count_column})
            Bad_bin_db['bad_counts'] = Bad_bin_db['ID'].map(bin_dict).fillna(0)

            result_folder = result_folder
            make_folder(result_folder)

            list_name = filename[file] + '_' + s + '_' + 'base_' + B 

            save_bin_db = pd.merge(Bad_bin_db, Good_bin_db, 
                           on = ["ID"], how = "inner")

            db_dict[list_name] = save_bin_db
    
            
# This section is to process db_list and save the files. Four types of files ["bad_counts", "good_counts", "error", "qscore"]
# at each cycle and bases will be saved. The file will be saved in the format of SNR bins as the row index and chastity bins
# as the column headings
chastity_name = ['chastity' + str(x) for x in range(1, len(bin_edges))]
SNR_name = ['SNR' + str(x) for x in range(1, len(bin_edges))]

for SNR_method in SNR_hist_bin.keys():
    for base in ["base_A", "base_T", "base_G", "base_C"]:
        L1 = [s for s in db_dict.keys() if SNR_method in s and base in s]
        L2 = []
        for file in L1:
            L2.append(db_dict[file])
        db = pd.concat(L2).groupby(by=["ID"]).sum().reset_index().sort_values(by=['ID'])
        db['ID'] = pd.Categorical(db['ID'], ordered=True, categories= ns.natsorted(db['ID'].unique()))
        db = db.sort_values('ID')
        db[['Cycle','chas_bin', 'SNR_bin']] = db.ID.str.split("_",expand=True,)
        db['error'] = db.bad_counts / (db.good_counts + db.bad_counts )
        db['qscore'] = round((10 * -np.log10(db.error)), 0)
        for cyc in selected_cycle:
            for name in ["bad_counts", "good_counts", "error", "qscore"]:
                db1 = db[db.Cycle == str(cyc)][name]
                n = len(bin_edges)-1
                outList = []
                for i in range(n, len(db1) + n, n):
                    outList.append(db1[i-n:i])

                db2 = pd.DataFrame(np.array(outList), columns=chastity_name,
                                  index = SNR_name)
                savefolder = result_folder + SNR_method + '/' + "cycle_" + str(cyc) + "_" +  base +  "/" 
                make_folder(savefolder)
                savefilename = savefolder + SNR_method + "_" + base + "_cycle_" + str(cyc) + "_" + name + '.csv'
                db2.to_csv(savefilename, sep=',', index=True)